# Set up Colour Science for Python with latest `develop` branch

In [2]:
!pip install -q colour-science

!pip uninstall -y colour-science
!if ! [ -d "colour" ]; then git clone https://github.com/colour-science/colour; fi
!if [ -d "colour" ]; then cd colour && git fetch && git checkout develop && cd ..; fi

import sys
sys.path.append('colour')

import colour
colour.utilities.describe_environment()

     |████████████████████████████████| 1.6MB 2.9MB/s 
Uninstalling colour-science-0.3.15:
  Successfully uninstalled colour-science-0.3.15
Cloning into 'colour'...
remote: Enumerating objects: 270, done.
remote: Counting objects: 100% (270/270), done.
remote: Compressing objects: 100% (186/186), done.
remote: Total 45133 (delta 170), reused 180 (delta 84), pack-reused 44863
Receiving objects: 100% (45133/45133), 110.60 MiB | 32.13 MiB/s, done.
Resolving deltas: 100% (34504/34504), done.
Already on 'develop'
Your branch is up to date with 'origin/develop'.
*                                                                             *
*   Interpreter :                                                             *
*       python : 3.6.9 (default, Jul 17 2020, 12:50:27)                       *
*                [GCC 8.4.0]                                                  *
*                                                                             *
*   colour-science.org :             

defaultdict(collections.OrderedDict,
            {'Interpreter': OrderedDict([('python',
                           '3.6.9 (default, Jul 17 2020, 12:50:27) \n[GCC 8.4.0]')]),
             'Runtime': OrderedDict([('imageio', '2.4.1'),
                          ('matplotlib', '3.2.2'),
                          ('networkx', '2.5'),
                          ('numpy', '1.18.5'),
                          ('pandas', '1.0.5'),
                          ('scipy', '1.4.1'),
                          ('six', '1.15.0')]),
             'colour-science.org': OrderedDict([('colour',
                           'v0.3.15-370-g1317e9df')])})

# Define Blackmagic Wide Gamut v4 colour space
# (not currently included in Colour)

In [3]:
import numpy as np

import colour
from colour.models import (
                           RGB_Colourspace,
                           normalised_primary_matrix
                          )
from colour.colorimetry import CCS_ILLUMINANTS

BMD_WIDE_GAMUT_V4_PRIMARIES = np.array([
    [0.7177, 0.3171],
    [0.2280, 0.8616],
    [0.1006, -0.0820],
])
"""
*BMD Wide Gamut V4* colourspace primaries.
As displayed on DaVinci Resolve Chromaticity scope.

BMD_WIDE_GAMUT_V4_PRIMARIES : ndarray, (3, 2)
"""

BMD_WIDE_GAMUT_V4_WHITEPOINT_NAME = 'D65'
"""
*BMD Wide Gamut V4* colourspace whitepoint name.

BMD_WIDE_GAMUT_V4_WHITEPOINT : unicode
"""

BMD_WIDE_GAMUT_V4_WHITEPOINT = (CCS_ILLUMINANTS[
    'CIE 1931 2 Degree Standard Observer'][BMD_WIDE_GAMUT_V4_WHITEPOINT_NAME])
"""
*BMD Wide Gamut V4* colourspace whitepoint.

BMD_WIDE_GAMUT_V4_WHITEPOINT : ndarray
"""

BMD_WIDE_GAMUT_V4_TO_XYZ_MATRIX = (normalised_primary_matrix(
    BMD_WIDE_GAMUT_V4_PRIMARIES, BMD_WIDE_GAMUT_V4_WHITEPOINT))
"""
*BMD Wide Gamut V4* colourspace to *CIE XYZ* tristimulus values matrix.

BMD_WIDE_GAMUT_V4_TO_XYZ_MATRIX : array_like, (3, 3)
"""

XYZ_TO_BMD_WIDE_GAMUT_V4_MATRIX = (
    np.linalg.inv(BMD_WIDE_GAMUT_V4_TO_XYZ_MATRIX))
"""
*CIE XYZ* tristimulus values to *BMD Wide Gamut V4* colourspace matrix.

XYZ_TO_BMD_WIDE_GAMUT_V4_MATRIX : array_like, (3, 3)
"""

RGB_COLOURSPACE_BMD_WIDE_GAMUT_V4 = RGB_Colourspace(
    'BMD Wide Gamut V4',
    BMD_WIDE_GAMUT_V4_PRIMARIES,
    BMD_WIDE_GAMUT_V4_WHITEPOINT,
    BMD_WIDE_GAMUT_V4_WHITEPOINT_NAME,
    BMD_WIDE_GAMUT_V4_TO_XYZ_MATRIX,
    XYZ_TO_BMD_WIDE_GAMUT_V4_MATRIX,
)

# Thresholds
Calculate `thresholds` to protect ColorChecker 24

In [5]:
from __future__ import division

import warnings

from colour.models import (
                           RGB_to_RGB,
                           RGB_COLOURSPACE_ACES2065_1,
                           RGB_COLOURSPACE_ACESCG,
                           RGB_COLOURSPACE_ALEXA_WIDE_GAMUT,
                           RGB_COLOURSPACE_CINEMA_GAMUT,
                           RGB_COLOURSPACE_RED_WIDE_GAMUT_RGB,
                           RGB_COLOURSPACE_S_GAMUT3,
                           RGB_COLOURSPACE_VENICE_S_GAMUT3,
                           RGB_COLOURSPACE_V_GAMUT
                          )
from colour.utilities import tsplit, tstack, dot_vector

# ColorChecker 24 values as per SMPTE 2065-1
CC24 = np.array([[0.11877, 0.08709, 0.05895],
                 [0.40002, 0.31916, 0.23736],
                 [0.18476, 0.20398, 0.31311],
                 [0.10901, 0.13511, 0.06493],
                 [0.26684, 0.24604, 0.40932],
                 [0.32283, 0.46208, 0.40606],
                 [0.38605, 0.22743, 0.05777],
                 [0.13822, 0.13037, 0.33703],
                 [0.30202, 0.13752, 0.12758],
                 [0.09310, 0.06347, 0.13525],
                 [0.34876, 0.43654, 0.10613],
                 [0.48655, 0.36685, 0.08061],
                 [0.08732, 0.07443, 0.27274],
                 [0.15366, 0.25692, 0.09071],
                 [0.21742, 0.07070, 0.05130],
                 [0.58919, 0.53943, 0.09157],
                 [0.30904, 0.14818, 0.27426],
                 [0.14901, 0.23378, 0.35939]])

CC24_AP1 = RGB_to_RGB(CC24, RGB_COLOURSPACE_ACES2065_1, RGB_COLOURSPACE_ACESCG)

ach = np.max(CC24_AP1, axis=-1)[..., np.newaxis]
dist = ((ach - CC24_AP1) / ach)
c, m, y = tsplit(dist)

print('Thresholds:')
print(np.max(c), np.max(m), np.max(y))

Thresholds:
0.814546172832 0.802945634483 0.87963185306


# Distance Limits
Calculate `distances` to map common camera encoding spaces to within AP1

In [9]:
maxLimits = np.array([[-1.0,  1.0,  1.0],
                      [ 1.0, -1.0,  1.0],
                      [ 1.0,  1.0, -1.0]])

spaces = [[RGB_COLOURSPACE_ALEXA_WIDE_GAMUT, 'CAT02'],
          [RGB_COLOURSPACE_BMD_WIDE_GAMUT_V4, 'Bradford'],
          [RGB_COLOURSPACE_CINEMA_GAMUT, 'CAT02'],
          [RGB_COLOURSPACE_RED_WIDE_GAMUT_RGB, 'Bradford'],
          [RGB_COLOURSPACE_S_GAMUT3, 'CAT02'],
          [RGB_COLOURSPACE_VENICE_S_GAMUT3, 'CAT02'],
          [RGB_COLOURSPACE_V_GAMUT, 'CAT02']]

bounds = []


for space, cat in spaces:
    vendorRGB = RGB_to_RGB(maxLimits,
                           RGB_COLOURSPACE_ACESCG,
                           space,
                           chromatic_adaptation_transform=cat)
    vendorHSV = colour.RGB_to_HSV(vendorRGB)

    for hsv in vendorHSV:
        hsv[1] = min(1.0, hsv[1])

    vendorRGB = colour.HSV_to_RGB(vendorHSV)

    bounds.append(RGB_to_RGB(vendorRGB,
                        space,
                        RGB_COLOURSPACE_ACESCG,
                        chromatic_adaptation_transform=cat))

ach = np.max(bounds, axis=-1)[..., np.newaxis]
dist = np.where(ach == 0.0, 0.0, (ach - bounds) / ach)

c, m, y = tsplit(dist)

print('\nDistance Limits:')
print(np.max(c) - 1, np.max(m) - 1, np.max(y) - 1)



Distance Limits:
0.147282233491 0.264290310772 0.312377540725
